# Generate GeoTIFF

In this notebook, the OpenStreetMap dump for Switzerland (i.e. `switzerland-latest-free.shp.zip`, downloaded from [Geofabrik](https://download.geofabrik.de/europe/switzerland.html)) is rasterized into a single GeoTIFF file.
Several channels are exported, with different meanings.

In [1]:
import math

import numpy as np
import pandas as pd

from pyproj import CRS, Transformer

import geopandas as gpd
import pyogrio

import cairo

import rasterio

from tqdm.auto import tqdm

In [2]:
pd.options.mode.copy_on_write = True

In [3]:
# File locations
input_path = "../data/switzerland-latest-free.shp.zip"
output_path = "../data/switzerland.tif"

# Size of a single pixel, in meters
step = 100

## Layer definitions

### Buildings

In [4]:
# papermill_description=LoadBuildings
path = f"{input_path}!gis_osm_buildings_a_free_1.shp"
building_df = pyogrio.read_dataframe(path)
building_df

,osm_id,code,fclass,name,type,geometry
0,4087332,1500,building,None,retail,"POLYGON ((9.37084 47.42154, 9.37132 47.42172, ..."
1,4418729,1500,building,Kulturzentrum Obere Mühle,None,"POLYGON ((8.62381 47.39540, 8.62395 47.39545, ..."
2,4418785,1500,building,Stadthaus Dübendorf,None,"POLYGON ((8.61789 47.39689, 8.61804 47.39702, ..."
3,4592724,1500,building,None,toilets,"POLYGON ((8.52785 47.37481, 8.52793 47.37485, ..."
4,4684660,1500,building,Airside Center,transportation,"POLYGON ((8.56039 47.45128, 8.56040 47.45136, ..."
...,...,...,...,...,...,...
2653328,1242981192,1500,building,None,None,"POLYGON ((9.56142 46.96688, 9.56145 46.96689, ..."
2653329,1242981193,1500,building,None,None,"POLYGON ((9.56027 46.96644, 9.56032 46.96646, ..."
2653330,1242981194,1500,building,None,None,"POLYGON ((9.56017 46.96637, 9.56024 46.96640, ..."
2653331,1242981195,1500,building,None,None,"POLYGON ((9.55978 46.96615, 9.55983 46.96618, ..."


In [5]:
building_df["fclass"].value_counts()

fclass
building    2653333
Name: count, dtype: int64

In [6]:
building_df["type"].value_counts()

type
residential        175200
house              121279
apartments         104124
garage              52192
detached            40937
                    ...  
clubhaus                1
storage                 1
production_hall         1
saw                     1
unterstand              1
Name: count, Length: 287, dtype: int64

### Land use

In [7]:
# papermill_description=LoadLandUse
path = f"zip://{input_path}!gis_osm_landuse_a_free_1.shp"
landuse_df = pyogrio.read_dataframe(path)
landuse_df

,osm_id,code,fclass,name,geometry
0,4251983,7201,forest,Brühlwald,"POLYGON ((8.70937 47.50023, 8.70943 47.50031, ..."
1,4260391,7201,forest,Wolfensberg,"POLYGON ((8.69771 47.52016, 8.69906 47.52173, ..."
2,4264293,7229,farmland,None,"POLYGON ((8.77179 47.48390, 8.77189 47.48430, ..."
3,4264597,7202,park,Unionsplatz,"POLYGON ((8.71885 47.48911, 8.71930 47.48934, ..."
4,4272831,7202,park,Parc Jardin Anglais,"POLYGON ((6.14998 46.20491, 6.15002 46.20503, ..."
...,...,...,...,...,...
303223,1242842548,7218,grass,None,"POLYGON ((9.30277 46.76950, 9.30315 46.76970, ..."
303224,1242847929,7218,grass,None,"POLYGON ((6.84861 46.84196, 6.84880 46.84223, ..."
303225,1242961769,7215,orchard,None,"POLYGON ((9.55454 46.96996, 9.55463 46.97022, ..."
303226,1242961780,7218,grass,None,"POLYGON ((9.55493 46.97050, 9.55498 46.97067, ..."


In [8]:
landuse_df["fclass"].value_counts().sort_index()

fclass
allotments            2277
cemetery              2331
commercial            1356
farmland             30205
farmyard             18450
forest               97639
grass                32261
heath                  804
industrial            4422
meadow               46564
military               510
nature_reserve         510
orchard               6499
park                  4064
quarry                1101
recreation_ground      650
residential          18023
retail                 424
scrub                26932
vineyard              8206
Name: count, dtype: int64

In [9]:
# Currently, we only extract forests
forest_df = landuse_df[landuse_df["fclass"] == "forest"]

### Roads

In [10]:
# papermill_description=LoadRoads
path = f"zip://{input_path}!gis_osm_roads_free_1.shp"
road_df = pyogrio.read_dataframe(path)
road_df

,osm_id,code,fclass,name,ref,oneway,maxspeed,layer,bridge,tunnel,geometry
0,78216,5131,motorway_link,None,A6,F,0,0,F,F,"LINESTRING (7.60120 46.74213, 7.60122 46.74212..."
1,78279,5114,secondary,Hohmadstrasse,None,F,50,0,F,F,"LINESTRING (7.62343 46.74946, 7.62335 46.74943..."
2,326729,5141,service,Wydengässli,None,B,0,0,F,F,"LINESTRING (7.83522 46.67479, 7.83518 46.67486..."
3,728458,5131,motorway_link,None,None,F,100,0,F,F,"LINESTRING (7.60285 46.74277, 7.60296 46.74300..."
4,728459,5131,motorway_link,None,None,F,60,0,F,F,"LINESTRING (7.60441 46.74321, 7.60438 46.74322..."
...,...,...,...,...,...,...,...,...,...,...,...
1636967,1243004338,5153,footway,None,None,B,0,0,F,F,"LINESTRING (8.17762 47.03455, 8.17762 47.03455)"
1636968,1243004339,5141,service,None,None,B,0,0,F,F,"LINESTRING (8.17764 47.03452, 8.17765 47.03450..."
1636969,1243004340,5153,footway,None,None,B,0,0,F,F,"LINESTRING (8.17795 47.03437, 8.17796 47.03440..."
1636970,1243004341,5153,footway,None,None,B,0,0,F,F,"LINESTRING (8.17812 47.03447, 8.17813 47.03448)"


In [11]:
road_df["fclass"].value_counts().sort_index()

fclass
bridleway            366
busway                 1
cycleway           17204
footway           285058
living_street       5772
motorway            8046
motorway_link       5706
path              189963
pedestrian          6887
primary            29507
primary_link         896
residential       179378
secondary          32103
secondary_link       582
service           373138
steps              51684
tertiary           44903
tertiary_link        389
track              77919
track_grade1       52843
track_grade2       89638
track_grade3       65598
track_grade4       29494
track_grade5       22804
trunk               2192
trunk_link          1067
unclassified       63808
unknown               26
Name: count, dtype: int64

In [12]:
road_df["oneway"].value_counts()

oneway
B    1562266
F      74518
T        188
Name: count, dtype: int64

In [13]:
road_df["tunnel"].value_counts()

tunnel
F    1619216
T      17756
Name: count, dtype: int64

In [14]:
# Ignore tunnels
road_df = road_df[road_df["tunnel"] == "F"]

In [15]:
# Estimate width based on type
# https://doc.arcgis.com/en/cityengine/latest/tutorials/tutorial-4-import-streets.htm
widths = {
    "primary": 8,
    "secondary": 7,
    "tertiary": 6,
    "motorway": 12,
    "trunk": 11,
    "road": 6,
    "residential": 5,
    "footway": 2,
    "cycleway": 2,
    "steps": 2,
}


def estimate_width(row):
    w = widths.get(row["fclass"], 4)
    if row["oneway"] == "T":
        w /= 2
    return w


road_df["width"] = road_df.apply(estimate_width, axis=1)

### Railways

In [16]:
# papermill_description=LoadRailways
path = f"zip://{input_path}!gis_osm_railways_free_1.shp"
railway_df = pyogrio.read_dataframe(path)
railway_df

,osm_id,code,fclass,name,layer,bridge,tunnel,geometry
0,2955839,6101,rail,None,0,F,F,"LINESTRING (7.49353 47.01410, 7.49345 47.01405..."
1,3847628,6101,rail,None,0,F,F,"LINESTRING (9.37312 47.42627, 9.37271 47.42592)"
2,3847633,6101,rail,None,0,F,F,"LINESTRING (9.30402 47.40330, 9.30224 47.40371..."
3,3847670,6101,rail,None,0,F,F,"LINESTRING (8.53644 47.40599, 8.53635 47.40585..."
4,3847672,6101,rail,None,0,F,F,"LINESTRING (8.52087 47.38423, 8.52062 47.38429..."
...,...,...,...,...,...,...,...,...
52596,1242172998,6101,rail,None,0,F,F,"LINESTRING (6.72229 46.48775, 6.72286 46.48773..."
52597,1242172999,6101,rail,None,0,F,F,"LINESTRING (6.72286 46.48773, 6.72317 46.48774..."
52598,1242294592,6101,rail,None,0,F,F,"LINESTRING (6.72229 46.48775, 6.72265 46.48772..."
52599,1242767423,6106,narrow_gauge,None,0,F,F,"LINESTRING (7.90225 46.63169, 7.90210 46.63170..."


In [17]:
railway_df["fclass"].value_counts().sort_index()

fclass
funicular              571
light_rail             220
miniature_railway      758
monorail                21
narrow_gauge          7824
rack                  1028
rail                 39341
subway                  82
tram                  2756
Name: count, dtype: int64

In [18]:
railway_df["tunnel"].value_counts()

tunnel
F    50413
T     2188
Name: count, dtype: int64

In [19]:
# Ignore tunnels
railway_df = railway_df[railway_df["tunnel"] == "F"]

In [20]:
# Filter out some classes
classes = {
    "funicular",
    "light_rail",
    # "miniature_railway",
    # "monorail",
    "narrow_gauge",
    "rack",
    "rail",
    "subway",
    "tram",
}
railway_df = railway_df[railway_df["fclass"].isin(classes)]

In [21]:
# Arbitrary select width, as none is provided
railway_df["width"] = 10

### Water and waterways

In [22]:
# papermill_description=LoadWater
path = f"zip://{input_path}!gis_osm_water_a_free_1.shp"
water_df = pyogrio.read_dataframe(path)
water_df

,osm_id,code,fclass,name,geometry
0,4245438,8200,water,Seelisbergsee,"POLYGON ((8.56781 46.95808, 8.56782 46.95815, ..."
1,4264525,8201,reservoir,Pfaffensprung-Stausee,"POLYGON ((8.60833 46.71275, 8.60834 46.71283, ..."
2,4264538,8201,reservoir,Göscheneralpsee,"POLYGON ((8.46877 46.64873, 8.46883 46.64891, ..."
3,4270536,8200,water,Lago della Piazza,"POLYGON ((8.56424 46.55724, 8.56429 46.55727, ..."
4,4270537,8200,water,Lago Scuro,"POLYGON ((8.56456 46.55869, 8.56460 46.55882, ..."
...,...,...,...,...,...
27367,1242544286,8200,water,None,"POLYGON ((9.57991 46.45703, 9.57996 46.45703, ..."
27368,1242596793,8200,water,None,"POLYGON ((8.40500 47.44256, 8.40500 47.44257, ..."
27369,1242683235,8200,water,None,"POLYGON ((7.90772 47.36562, 7.90772 47.36563, ..."
27370,1242808257,8200,water,None,"POLYGON ((9.26284 46.66809, 9.26293 46.66816, ..."


In [23]:
water_df["fclass"].value_counts()

fclass
water        19607
wetland       4106
riverbank     1443
reservoir     1155
glacier       1052
dock             9
Name: count, dtype: int64

In [24]:
# Filter out some classes
classes = {
    # "dock",
    # "glacier",
    "reservoir",
    "river",
    # "riverbank",
    "water",
    "wetland",
}
water_df = water_df[water_df["fclass"].isin(classes)]

In [25]:
# papermill_description=LoadWaterways
path = f"zip://{input_path}!gis_osm_waterways_free_1.shp"
waterway_df = gpd.read_file(path)
waterway_df

,osm_id,code,fclass,width,name,geometry
0,4223448,8101,river,0,Äussere Aare,"LINESTRING (7.63280 46.75648, 7.63245 46.75650..."
1,4224611,8103,canal,0,None,"LINESTRING (7.63540 46.74993, 7.63267 46.75318..."
2,4224616,8101,river,0,Innere Aare,"LINESTRING (7.63280 46.75648, 7.63243 46.75701..."
3,4245442,8101,river,0,Reuss,"LINESTRING (8.63822 46.83041, 8.63796 46.83080..."
4,4245448,8101,river,0,Reuss,"LINESTRING (8.66776 46.76685, 8.66829 46.76706..."
...,...,...,...,...,...,...
149055,1242544312,8102,stream,0,None,"LINESTRING (9.54777 46.45467, 9.54773 46.45463)"
149056,1242752721,8102,stream,0,None,"LINESTRING (6.87959 47.20258, 6.87968 47.20238..."
149057,1242842545,8102,stream,0,None,"LINESTRING (9.31977 46.75822, 9.31910 46.75868..."
149058,1242842546,8102,stream,0,None,"LINESTRING (9.31788 46.75696, 9.31757 46.75722..."


In [26]:
waterway_df["fclass"].value_counts()

fclass
stream    140718
drain       3867
river       2508
canal       1967
Name: count, dtype: int64

In [27]:
# Override line widths
# TODO fine-tune these values
widths = {
    "stream": 10,
    "drain": 10,
    "river": 50,
    "canal": 50,
}
assert waterway_df["fclass"].isin(widths).all()
waterway_df["width"] = waterway_df["fclass"].map(widths)

## Rasterize layers

In [28]:
# Select geometry for each GeoTIFF channel
dfs = {
    "building": building_df,
    "road": road_df,
    "railway": railway_df,
    "water": water_df,
    "waterway": waterway_df,
    "forest": forest_df,
}

In [29]:
# Using LV03
# https://epsg.io/21781
# TODO should probably use LV95 instead?
crs = CRS.from_epsg(21781)
crs

<Projected CRS: EPSG:21781>
Name: CH1903 / LV03
Axis Info [cartesian]:
- Y[east]: Easting (metre)
- X[north]: Northing (metre)
Area of Use:
- name: Liechtenstein; Switzerland.
- bounds: (5.96, 45.82, 10.49, 47.81)
Coordinate Operation:
- name: Swiss Oblique Mercator 1903M
- method: Hotine Oblique Mercator (variant B)
Datum: CH1903
- Ellipsoid: Bessel 1841
- Prime Meridian: Greenwich

In [30]:
# Use official bounds
transformer = Transformer.from_crs(crs.geodetic_crs, crs, always_xy=True)
bounds = crs.area_of_use.bounds
x_min, y_min, x_max, y_max = transformer.transform_bounds(*bounds)
x_min, y_min, x_max, y_max

(485014.011782823, 74129.69800843272, 837016.8922540342, 299782.7670088512)

In [31]:
# Round bounds accordingly, so that we are aligned on a multiple of the step
x_min = math.floor(x_min / step) * step
y_min = math.floor(y_min / step) * step
x_max = math.ceil(x_max / step) * step
y_max = math.ceil(y_max / step) * step
x_min, y_min, x_max, y_max

(485000, 74100, 837100, 299800)

In [32]:
# Raster shape
width = (x_max - x_min) // step
height = (y_max - y_min) // step
height, width

(2257, 3521)

In [33]:
# papermill_description=ConvertCRS
dfs = {name: df.to_crs(crs) for name, df in dfs.items()}

In [34]:
# papermill_description=Rasterize


def transform(x, y):
    x = (x - x_min) / step
    y = (y_max - y) / step
    return x, y


def add_path(ctx, coords, close):
    x, y = coords[0]
    x, y = transform(x, y)
    ctx.move_to(x, y)

    for x, y in coords[1:]:
        x, y = transform(x, y)
        ctx.line_to(x, y)

    if close:
        ctx.close_path()


def add_polygon(ctx, polygon):
    add_path(ctx, polygon.exterior.coords, close=True)
    for interior in polygon.interiors:
        add_path(ctx, interior.coords, close=True)


def rasterize(df):
    # Use Cairo to rasterize geometry with fractional coverage
    # Note: `rasterio.features.rasterize` only support binary coverage
    surface = cairo.ImageSurface(cairo.FORMAT_RGB96F, width, height)
    ctx = cairo.Context(surface)
    ctx.set_source_rgb(1.0, 1.0, 1.0)
    ctx.set_fill_rule(cairo.FILL_RULE_EVEN_ODD)

    for index, geometry in tqdm(df.geometry.items(), total=len(df)):
        match geometry.geom_type:
            case "Polygon":
                add_polygon(ctx, geometry)
                ctx.fill()

            case "MultiPolygon":
                for polygon in geometry.geoms:
                    add_polygon(ctx, polygon)
                ctx.fill()

            case "LineString":
                ctx.set_line_width(df.loc[index, "width"])
                add_path(ctx, geometry.coords, close=False)
                ctx.stroke()

            case _:
                raise NotImplementedError

    # Extract raster as array
    buffer = surface.get_data()
    array = np.frombuffer(buffer, dtype=np.float32).reshape(height, width, 3)

    # Only keep a single channel, as the rendered image is grayscale anyway
    array = array[:, :, 0]

    return array


# Rasterize each layer separately
arrays = {}
for name, df in tqdm(dfs.items()):
    array = rasterize(df)
    arrays[name] = array

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/2653333 [00:00<?, ?it/s]

  0%|          | 0/1619216 [00:00<?, ?it/s]

  0%|          | 0/49709 [00:00<?, ?it/s]

  0%|          | 0/24868 [00:00<?, ?it/s]

  0%|          | 0/149060 [00:00<?, ?it/s]

  0%|          | 0/97639 [00:00<?, ?it/s]

In [35]:
# papermill_description=ExportGeoTiff
metadata = {
    "width": width,
    "height": height,
    "count": len(arrays),
    "crs": crs,
    "transform": rasterio.transform.from_origin(x_min, y_max, step, step),
    "dtype": np.uint8,
}
with rasterio.open(output_path, "w", **metadata) as dst:
    for i, (name, array) in enumerate(arrays.items(), start=1):
        dst.write(array * 255, i)
        dst.set_band_description(i, name)